In [3]:
import pandas as pd

In [7]:
data = pd.read_csv('data_observed.csv')
data_observed = data[['X', 'Y', 'T']]
X = data_observed['X']
T = data_observed['T']
Y = data_observed['Y']


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# Reshape X to have shape (n_samples, n_features)
X_reshaped = X.reshape(-1, 1)  # Reshape to (n_samples, 1)
T_reshaped = T.reshape(-1, 1)  # Reshape to (n_samples, 1)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_reshaped, dtype=torch.float32)
T_tensor = torch.tensor(T_reshaped, dtype=torch.float32)

# Define the Propensity Score Model (Neural Network)
class PropensityNN(nn.Module):
    def __init__(self):
        super(PropensityNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 16),  # Input dimension is 1
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # Output probability of T=1
        )

    def forward(self, x):
        return self.fc(x)

# Initialize model, loss function, and optimizer
model = PropensityNN()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    pred_T = model(X_tensor)
    loss = criterion(pred_T, T_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Compute Propensity Scores
with torch.no_grad():
    e_X = model(X_tensor).numpy()

# Compute Inverse Propensity Weights
IPS_weights = np.where(T_reshaped == 1, 1 / e_X, 1 / (1 - e_X))

AttributeError: 'Series' object has no attribute 'reshape'